In [1]:
# Agrega as matrículas de escolas aos hexágonos h3 res 09

# carregar bibliotecas
suppressPackageStartupMessages(library('tidyverse'))
suppressPackageStartupMessages(library('tidylog'))
suppressPackageStartupMessages(library('sf'))
suppressPackageStartupMessages(library('mapview'))

In [2]:
# Estrutura de pastas e arquivos
pasta_dados       <- "../../yellow_dados"
dados_originais   <- sprintf("%s/00_dados_originais", pasta_dados)
pasta_ipea        <- sprintf("%s/IPEA", dados_originais)
pasta_aop_optimum <- sprintf("%s/13_aop_optimum", pasta_dados)
pasta_opaop_dados <- sprintf("%s/02_dados_pop_mat", pasta_aop_optimum)

In [3]:
# IPEA - Shape de grid hexagonal (resolução 9)
hexagonos <- sprintf('%s/aop_hex_grid_v2.gpkg', pasta_ipea)
hexagonos <- read_sf(hexagonos) %>% filter(abbrev_muni == 'spo') %>% select(id_hex)
# mapview(hexagonos)

# Censo escolar 2019 com matrículas
escolas <- sprintf('%s/matriculas_censo_escolar_2019_georref.gpkg', pasta_opaop_dados)
escolas <- st_read(escolas)
escolas <- escolas %>% select(CO_ENTIDADE, QT_MAT_MED, QT_MAT_BAS_15_17, geom)
head(escolas, 2)

# Caso projeções entre os dados do Censo Escolar e do grid de hexágonos sejam 
# diferentes, reprojetar para o CRS do primeiro
if (st_crs(hexagonos) != st_crs(escolas)) {
  hexagonos <- hexagonos %>% st_transform(crs = st_crs(escolas))
}
head(hexagonos, 2)

filter: removed 374,971 rows (96%), 15,059 rows remaining

select: dropped 3 variables (abbrev_muni, name_muni, code_muni)



Reading layer `matriculas_censo_escolar_2019_georref' from data source `/media/livre/HDINT1TB/Base_GtsRegionais/GitLab/yellow_dados/13_aop_optimum/02_dados_pop_mat/matriculas_censo_escolar_2019_georref.gpkg' using driver `GPKG'
Simple feature collection with 1376 features and 20 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -46.80379 ymin: -23.90824 xmax: -46.37497 ymax: -23.40061
Geodetic CRS:  WGS 84


select: dropped 17 variables (NO_ENTIDADE, TP_DEPENDENCIA, TP_CATEGORIA_ESCOLA_PRIVADA, TP_LOCALIZACAO, DS_ENDERECO, …)

Warning message in fun(libname, pkgname):
“rgeos: versions of GEOS runtime 3.9.1-CAPI-1.14.2
and GEOS at installation 3.9.1dev-CAPI-1.14.1differ”
Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson



,CO_ENTIDADE,QT_MAT_MED,QT_MAT_BAS_15_17,geom
,<chr>,<dbl>,<dbl>,<POINT [°]>
1,35000024,905,779,POINT (-46.75938 -23.40471)
2,35000036,946,983,POINT (-46.6984 -23.46947)


id_hex,geom
<chr>,<POLYGON [°]>
89a81009a8bffff,POLYGON ((-46.43596 -23.586...
89a8108dca7ffff,POLYGON ((-46.77886 -23.897...


In [4]:
# Fazer interseção entre as escolas e os hexágonos
escolas_hex <- st_intersection(escolas, hexagonos)
head(escolas_hex)

although coordinates are longitude/latitude, st_intersection assumes that they are planar

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”


,CO_ENTIDADE,QT_MAT_MED,QT_MAT_BAS_15_17,id_hex,geom
,<chr>,<dbl>,<dbl>,<chr>,<POINT [°]>
51,35000929,781,714,89a8100d9d7ffff,POINT (-46.62531 -23.50148)
432,35046059,452,512,89a81015653ffff,POINT (-46.69049 -23.78288)
32,35000589,23,36,89a81039653ffff,POINT (-46.6838 -23.48312)
1309,35915725,781,717,89a81007383ffff,POINT (-46.74247 -23.65847)
607,35105971,119,109,89a810052cfffff,POINT (-46.68862 -23.61262)
355,35037655,819,694,89a8103921bffff,POINT (-46.6873 -23.46018)


In [5]:
# Agrupar matrículas por hexágono
escolas_hex <- 
  escolas_hex %>% 
  st_drop_geometry() %>% 
  group_by(id_hex) %>% 
  summarise(matriculas_ensino_medio = sum(QT_MAT_MED),
            matriculas_idades_15_17 = sum(QT_MAT_BAS_15_17))

head(escolas_hex)

group_by: one grouping variable (id_hex)

summarise: now 1,241 rows and 3 columns, ungrouped



id_hex,matriculas_ensino_medio,matriculas_idades_15_17
<chr>,<dbl>,<dbl>
89a81000027ffff,1051,1048
89a8100002bffff,357,369
89a81000077ffff,350,317
89a8100008fffff,309,325
89a81000133ffff,128,116
89a81000163ffff,73,69


In [6]:
# Conferência de valores
sum(escolas$QT_MAT_MED)
sum(escolas$QT_MAT_BAS_15_17)
sum(escolas_hex$matriculas_ensino_medio)
sum(escolas_hex$matriculas_idades_15_17)

[1] 387984

[1] 374710

[1] 387984

[1] 374710

In [7]:
# Gravar resultados
out_file <- sprintf('%s/matriculas_censo_escolar_2019_por_hexagono.csv', pasta_aop_optimum)
write_delim(escolas_hex, out_file, delim = ';')

In [8]:
# Juntar ao grid de hexágonos para exportação
hex_mat <- 
  left_join(hexagonos, escolas_hex, by = 'id_hex') %>% 
  replace(is.na(.), 0) %>% 
  relocate(geom, .after = last_col())

head(hex_mat, 2)

left_join: added 2 columns (matriculas_ensino_medio, matriculas_idades_15_17)

           > rows only in x   13,818

           > rows only in y  (     0)

           > matched rows      1,241

           >                 ========

           > rows total       15,059

relocate: columns reordered (id_hex, matriculas_ensino_medio, matriculas_idades_15_17, geom)



id_hex,matriculas_ensino_medio,matriculas_idades_15_17,geom
<chr>,<dbl>,<dbl>,<POLYGON [°]>
89a81009a8bffff,0,0,POLYGON ((-46.43596 -23.586...
89a8108dca7ffff,0,0,POLYGON ((-46.77886 -23.897...


In [9]:
# Gravar resultados
out_file2 <- sprintf('%s/matriculas_censo_escolar_2019_por_hexagono.gpkg', pasta_opaop_dados)
st_write(hex_mat, out_file2, driver = 'GPKG', append = FALSE)

Writing layer `matriculas_censo_escolar_2019_por_hexagono' to data source `../../yellow_dados/13_aop_optimum/02_dados_pop_mat/matriculas_censo_escolar_2019_por_hexagono.gpkg' using driver `GPKG'
Writing 15059 features with 3 fields and geometry type Polygon.
